# **1321. Restaurant Growth**
``` console
Table: Customer

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |
+---------------+---------+
In SQL,(customer_id, visited_on) is the primary key for this table.
This table contains data about customer transactions in a restaurant.
visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
amount is the total paid by a customer.
 

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
+-------------+--------------+--------------+-------------+
| customer_id | name         | visited_on   | amount      |
+-------------+--------------+--------------+-------------+
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 
+-------------+--------------+--------------+-------------+

Output: 
+--------------+--------------+----------------+
| visited_on   | amount       | average_amount |
+--------------+--------------+----------------+
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |
+--------------+--------------+----------------+

Explanation: 
1st moving average from 2019-01-01 to 2019-01-07 has an average_amount of (100 + 110 + 120 + 130 + 110 + 140 + 150)/7 = 122.86
2nd moving average from 2019-01-02 to 2019-01-08 has an average_amount of (110 + 120 + 130 + 110 + 140 + 150 + 80)/7 = 120
3rd moving average from 2019-01-03 to 2019-01-09 has an average_amount of (120 + 130 + 110 + 140 + 150 + 80 + 110)/7 = 120
4th moving average from 2019-01-04 to 2019-01-10 has an average_amount of (130 + 110 + 140 + 150 + 80 + 110 + 130 + 150)/7 = 142.86
```

**Solutions**

In [ ]:
-- Write your PostgreSQL query statement below
WITH DailySales AS (
SELECT  visited_on , SUM(amount) AS amount
FROM Customer
GROUP BY visited_on ),
moving_avgs as(
SELECT 
visited_on,
SUM(amount) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)AS amount ,
ROUND(AVG(amount) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW),2) AS average_amount,
MIN(visited_on) OVER() as min_date
FROM DailySales)

SELECT visited_on, amount, average_amount 
FROM moving_avgs
WHERE (visited_on - min_date) >= 6

In [ ]:
# Write your MySQL query statement below
SELECT
    visited_on,
    (
        SELECT SUM(amount)
        FROM customer
        WHERE visited_on BETWEEN DATE_SUB(c.visited_on, INTERVAL 6 DAY) AND c.visited_on
    ) AS amount,
    ROUND(
        (
            SELECT SUM(amount) / 7
            FROM customer
            WHERE visited_on BETWEEN DATE_SUB(c.visited_on, INTERVAL 6 DAY) AND c.visited_on
        ),
        2
    ) AS average_amount
FROM customer c
WHERE visited_on >= (
        SELECT DATE_ADD(MIN(visited_on), INTERVAL 6 DAY)
        FROM customer
    )
GROUP BY visited_on;

In [ ]:
select distinct visited_on,
        sum(amount) over w as amount,
        round((sum(amount) over w)/7, 2) as average_amount
    from customer
    WINDOW w AS ( 
            order by visited_on
            range between interval 6 day PRECEDING and current row
    )
    OFFSET 6

In [ ]:
-- Write your PostgreSQL query statement below

WITH temp as 
(SELECT DISTINCT visited_on FROM Customer 
WHERE (visited_on - (SELECT MIN(visited_on) FROM Customer)) >= 6),

temp2 AS 
(SELECT c.*,t.visited_on as vo FROM Customer c JOIN temp t ON t.visited_on - c.visited_on BETWEEN 0 AND 6)

SELECT vo as visited_on,SUM(amount) AS amount, ROUND(SUM(amount) / 7::numeric, 2) AS average_amount FROM temp2
GROUP BY vo

In [ ]:
WITH virtualcustomer
     AS (SELECT visited_on,
                SUM(amount) AS vc_amount
         FROM   customer
         GROUP  BY visited_on)
SELECT visited_on,
       amount,
       average_amount
FROM   (SELECT visited_on,
               SUM(vc_amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 preceding AND CURRENT ROW) AS amount,
               Round(Avg(vc_amount) OVER (ORDER BY visited_on ROWS BETWEEN 6 preceding AND CURRENT ROW), 2) AS average_amount,
               ROW_NUMBER() OVER (ORDER BY visited_on) AS rn
        FROM   virtualcustomer) AS t
WHERE  rn > 6
ORDER  BY visited_on;